### Code that was written to process LAS files, extract the relevant data, and generate Excel files with two tabs: one for the LAS data and one for the user definition. 

In [ ]:
import pandas as pd

# Function to read LAS file and process the data
def process_las_file(las_file_path, export_filename):
    with open(las_file_path, 'r', encoding='ISO-8859-1') as file:
        las_content = file.readlines()

    # Extract the filename based on the first hashtag line in the LAS file
    export_filename = None
    for line in las_content:
        if line.startswith('#'):
            export_filename = line.strip().replace('# ', '').replace(' ', '_') + ".xlsx"
            break

    # Set a fallback filename if none found
    if not export_filename:
        export_filename = "LAS_data_export.xlsx"

    # Extract the data section starting after the first occurrence of ~User_Data
    las_data_section_start = False
    las_data_rows = []
    for line in las_content:
        if line.startswith('~User_Data'):
            las_data_section_start = True
        elif las_data_section_start and ',' in line:
            las_data_rows.append(line.strip().split(','))

    # Define the appropriate column names (adjust this for your file)
    column_names = [
        'Depth', 'Sample ID', 'Sample Type', 'Vendor', 'Sample Condition', 'Saturation Fluid', 
        'Saturation', 'Porosity', 'Effective Confining Stress', 'Effective Axial Stress', 
        'Bulk Density', 'Grain Density', 'Saturation Fluid Density', 'Compressional Velocity',
        'Compressional Transit Time', 'Shear Velocity', 'Shear Transit Time', 'Dynamic Modulus Shear', 
        'Dynamic Modulus Young\'s', 'Dynamic Modulus Bulk', 'Poisson\'s Ratio', 'Pore Pressure',
        'Lithology', 'Notes'
    ]

    # Convert the extracted rows into a DataFrame using the column names
    df_las = pd.DataFrame(las_data_rows[1:], columns=column_names)

    # Apply "See second tab" to the second row of the 'Notes' column
    df_las['Notes'] = None  # Reset the entire 'Notes' column
    df_las.loc[1, 'Notes'] = 'See second tab'  # Add the note to the second row

    # Define the unique user definition content for the second tab
    user_definition_data = [
        ['DPTH', 'feet', 'Depth'],
        ['SMPL_ID', '.', 'Sample ID {S}'],
        ['SMPL_TYP', '.', 'Sample Type {S}'],
        ['VNDR', '.', 'Data Vendor {S}'],
        ['SMPL_CNDTN', '.', 'Sample Condition {S}'],
        ['STRTN_FLD', '.', 'Saturation Fluid {S}'],
        ['STRTN', 'fraction', 'Saturation'],
        ['PRSTY', 'percent', 'Porosity'],
        ['EFF_CNFNG_STRS', 'psi', 'Effective Confining Stress'],
        ['EFF_AXL_STRS', 'psi', 'Effective Axial Stress'],
        ['BLK_DNS', 'gm/cm(3)', 'Bulk Density'],
        ['GRN_DNS', 'gm/cm(3)', 'Grain Density'],
        ['STRTN_FLD_DNS', 'gm/cm(3)', 'Saturation Fluid Density'],
        ['CMPRSNL_VLCTY', 'ft/s', 'Compressional Velocity'],
        ['CMPRSNL_TRNST_TM', 'µs/ft', 'Compressional Transit Time'],
        ['SHR_VLCTY', 'ft/s', 'Shear Velocity'],
        ['SHR_TRNST_TM', 'µs/ft', 'Shear Transit Time'],
        ['DYNMC_MDL_SHR', 'psi', 'Dynamic Modulus Shear'],
        ['DYNMC_MDL_YNGS', 'psi', 'Dynamic Modulus Young\'s'],
        ['DYNMC_MDL_BLK', 'psi', 'Dynamic Modulus Bulk'],
        ['PSNS_RTO', '.', 'Poisson\'s Ratio'],
        ['PR_PRS', 'psi', 'Pore Pressure'],
        ['LTHLGY', '.', 'Lithology {S}'],
        ['NOTES', '.', 'Notes {S}']
    ]

    # Create a DataFrame for the unique user definition
    df_user_definition = pd.DataFrame(user_definition_data, columns=['Column Mnemonic', 'Column Units', 'Column Description'])

    # Save both the LAS data and the unique User Definition to the Excel file
    export_path = f'{export_filename}'
    with pd.ExcelWriter(export_path, engine='openpyxl') as writer:
        # Save the LAS data to the first sheet
        df_las.to_excel(writer, sheet_name='LAS Data', index=False)
        
        # Save the unique User Definition to the second sheet
        df_user_definition.to_excel(writer, sheet_name='User_Definition', index=False)

    return export_path

# Example usage
las_file_path = 'your_las_file.las'  # Replace with your LAS file path
export_filename = 'exported_file.xlsx'  # Desired export filename
process_las_file(las_file_path, export_filename)
